In [2]:
!pip install transformers
!pip install torch

In [3]:
import csv
import torch
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer

tqdm.pandas()
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/readmissions_data.csv")
data = data.drop(columns = ["Unnamed: 0"])

In [6]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [7]:
#summarizer1 = pipeline("summarization", model="facebook/bart-large-cnn")
#print(summarizer1(data.iloc[1]["Report1"], max_length=130, min_length=30, do_sample=False, truncation=True))
#print(summarizer1(data.iloc[1]["Report2"], max_length=130, min_length=30, do_sample=False, truncation=True))

In [8]:
data = data[["SUBJECT_ID", "Report1", "Report2"]]
subset = data[:125]
summarizer_final = pipeline("summarization", model = "facebook/bart-large-cnn")
subset["Report1"] = subset["Report1"].progress_apply(lambda i: (summarizer_final(i, max_length = 130, min_length = 30, do_sample = False, truncation = True))[0]["summary_text"])
subset["Report2"] = subset["Report2"].progress_apply(lambda i: (summarizer_final(i, max_length = 130, min_length = 30, do_sample = False, truncation = True))[0]["summary_text"])

100%|██████████| 125/125 [31:39<00:00, 15.20s/it]


In [9]:
subset

,SUBJECT_ID,Report1,Report2
0,36,A 69-year-old male recently had a CABG. He fel...,69-year-old male with recent history of CABG p...
1,41,The patient is a 56-year-old male with a histo...,56-year-old man with newly-diagnosed low-grade...
2,357,64 year old male with Hx of cirrhosis [**1-20*...,The patient was a 64-year-old male with fever....
3,362,An 85-year-old female with a history of hypert...,"Hematoma, AFIB (WITHOUT ANTOCOAGULATION SECOND..."
4,364,The patient is a 71-year-old male with a histo...,71 year old man with COPD and severe hemoptysi...
...,...,...,...
120,591,A 77-year-old man was admitted to the hospital...,77 year old s/p cabg c/dyspnea. Small left api...
121,594,Patient is an unfortunate 80-year-old female w...,Patient trached with 6.0 shiley. Vent settings...
122,595,The patient is a 44-year-old female who is pos...,45 year old woman s/p R vertebral art aneurysm...
123,605,The patient is an 85 year old nursing home res...,86-year-old woman with acute renal failure and...


In [10]:
#(subset["Report1"].iloc[0])[0]["summary_text"]

In [11]:
subset.to_csv("/content/drive/My Drive/Colab Notebooks/summarized_data1.csv", encoding='utf-8')